# MoleculeACE - ChEMBL cliff evaluation

Finally, once our models are trained on MoleculeACE data, we evaluate them on the accompanying MoleculeACE test data.

* ChEMBL234 - Dopamine D3 receptor
* ChEMBL4203 - Dual specificity protein kinase
* ChEMBL2047 - Farnesoid X receptor
* ChEMBL4616 - Ghrelin receptor
* ChEMBL264 - Histamine H3 receptor
* ChEMBL2835 - Janus kinase 1
* ChEMBL4792 - Orexin receptor 2

## Setup

### Colab mount

In [1]:
import os.path

try:
    from google.colab import drive
    drive.mount('/content/drive')
    _home = 'drive/MyDrive/tlacamr'
except ImportError:
    _home = '~'
finally:
    project_root = os.path.join(_home, 'tlacamr')

print(project_root)
%cd $project_root

Mounted at /content/drive
drive/MyDrive/tlacamr/tlacamr
/content/drive/MyDrive/tlacamr/tlacamr


### Package install

In [2]:
%%capture
!pip install .
##once repo is public: !pip install git+https://github.com/cmvcordova/my-repo


### Imports

#### Main evaluation function

In [3]:
## general
import torch
import numpy as np
import os
import logging
import glob
import csv
from lightning import Trainer
from torch.utils.data import DataLoader

## acsuite imports
from src.models.encoding_module import ACAModule
from src.models.property_prediction_module import ACAPPModule
from src.data.property_prediction.moleculeace_dataset import MoleculeACEDataset
from src.data.property_prediction.moleculeace_datamodule import MoleculeACEDataModule
## molfeat related
import datamol as dm
from molfeat.calc import FPCalculator
from molfeat.trans import MoleculeTransformer

## metrics
from MoleculeACE import calc_rmse, calc_cliff_rmse
from sklearn.metrics import roc_auc_score, mean_squared_error

pretrained_model_dir = os.path.join("src", "models", "pretrained", "property_prediction")

def evaluate_datasets(datasets, model_type=None, fingerprint_length=2048):
    results = {}

    for dataset_info in datasets:
        dataset_name = dataset_info["name"]
        task = dataset_info["task"]

        task_key = f"{dataset_name}_{task}"
        molfeat_featurizer = MoleculeTransformer(FPCalculator('ecfp', length=fingerprint_length, radius=4))
        # Instantiate MoleculeACEDataset for the test split
        test_dataset = MoleculeACEDataset(dataset_name=dataset_name, data_split='test', task=task, molfeat_featurizer=molfeat_featurizer)
        # Create DataLoader for the test split
        test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

        # Find the correct checkpoint path using glob
        # Adjust the path to include the model_type directory and wildcard pattern for subdirectories
        checkpoint_pattern = os.path.join(pretrained_model_dir, task, model_type, dataset_name, "checkpoints", "epoch=*.ckpt")
        checkpoint_files = glob.glob(checkpoint_pattern)
        checkpoint_files.sort()  # Sort the files to get the latest
        if checkpoint_files:
            checkpoint_path = checkpoint_files[-1]  # Select the last checkpoint, which is the latest
            model = ACAPPModule.load_from_checkpoint(checkpoint_path)  # Load the model from the checkpoint
            print(f"Loaded checkpoint from {checkpoint_path}")
            print(f"Evaluating {dataset_name} ({task})")

            trainer = Trainer()

            # Perform prediction using the DataLoader
            predictions = trainer.predict(model, dataloaders=test_dataloader)
            y_pred = torch.cat(predictions).numpy()

            # Extract true labels directly from the dataset
            y_true = np.array(test_dataset.y)

            # Evaluate the model
            if task == "regression":
                test_cliffs = MoleculeACEDataset(dataset_name=dataset_name, data_split='test', task='classification').y
                rmse = calc_rmse(y_true, y_pred)
                cliff_rmse = calc_cliff_rmse(y_test_pred=y_pred,
                                             y_test=y_true,
                                             cliff_mols_test=test_cliffs)
                results[task_key] = {"RMSE": rmse, "Cliff_RMSE": cliff_rmse}
            elif task == "classification":
                if y_pred.ndim > 1 and y_pred.shape[1] > 1:
                    # Use the second column for AUROC calculation
                    y_pred_positive_class = y_pred[:, 1]
                else:
                    # Directly use y_pred if it's already the probability of the positive class
                    y_pred_positive_class = y_pred.flatten()

                auroc = roc_auc_score(y_true, y_pred_positive_class)
                results[task_key] = {"AUROC": auroc}

            print(f"Evaluation completed for {dataset_name} ({task})")
        else:
            print(f"No checkpoint found for {dataset_name} ({task})")
            continue
    return results

#### Print, utils

In [4]:
def write_results_to_csv(results_dict, file_path):
    headers = ['Model', 'Dataset', 'Task', 'Metric', 'Value']
    mode = 'w' if not os.path.exists(file_path) else 'a'
    already_exists = os.path.exists(file_path)

    with open(file_path, mode, newline='') as file:
        writer = csv.writer(file)
        if not already_exists:
            writer.writerow(headers)  # Write headers only if the file did not exist

        for model_name, results in results_dict.items():
            for task_key, metrics in results.items():
                dataset, task = task_key.rsplit('_', 1)
                for metric, value in metrics.items():
                    writer.writerow([model_name, dataset, task, metric, value])

## Evaluation

### Declare eval datasets

In [5]:
# Declare datasets. Analysis categories in comments.

chembl_datasets = [
    # DATASET SIZE:
    ## Highest amount of molecules
    {"name": "CHEMBL234_Ki", "task": "classification"},
    {"name": "CHEMBL234_Ki", "task": "regression"},
    ## Lowest amount of molecules
    {"name": "CHEMBL2835_Ki", "task": "classification"},
    {"name": "CHEMBL2835_Ki", "task": "regression"},
    # AMOUNT OF CLIFF PARTNERS:
    ## Highest amount
    {"name": "CHEMBL4203_Ki", "task": "classification"},
    {"name": "CHEMBL4203_Ki", "task": "regression"},
    ## Lowest amount
    {"name": "CHEMBL4616_EC50", "task": "classification"},
    {"name": "CHEMBL4616_EC50", "task": "regression"},
    # HIGHEST SIMILARITY ACROSS METRICS:
    ## Highest mean scaffold similarity
    {"name": "CHEMBL264_Ki", "task": "classification"},
    {"name": "CHEMBL264_Ki", "task": "regression"},
    ## Highest substructure similarity
    {"name": "CHEMBL4792_Ki", "task": "classification"},
    {"name": "CHEMBL4792_Ki", "task": "regression"},
    ## Highest SMILES similarity
    {"name": "CHEMBL2047_EC50", "task": "classification"},
    {"name": "CHEMBL2047_EC50", "task": "regression"}
]

### Joint 1024 (Input 1024*2)

In [6]:
pt_joint_1024_results = evaluate_datasets(chembl_datasets, "pt_joint_1024", fingerprint_length=1024)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_joint_1024/CHEMBL234_Ki/checkpoints/epoch=10-step=407.ckpt
Evaluating CHEMBL234_Ki (classification)


INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_joint_1024/CHEMBL2047_EC50/checkpoints/epoch=11-step=84.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


### Joint Autoencoder 1024 (Input 1024*2)

In [7]:
pt_joint_recon_1024_results = evaluate_datasets(chembl_datasets, "pt_joint_recon_1024", fingerprint_length=1024)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_joint_recon_1024/CHEMBL234_Ki/checkpoints/epoch=80-step=2997.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_joint_recon_1024/CHEMBL2047_EC50/checkpoints/epoch=3-step=28.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


### Siamese classification - Manhattan distance

In [8]:
pt_siamese_manhattan_results = evaluate_datasets(chembl_datasets, "pt_siamese_manhattan", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_siamese_manhattan/CHEMBL234_Ki/checkpoints/epoch=11-step=444.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_siamese_manhattan/CHEMBL2047_EC50/checkpoints/epoch=3-step=28.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


### Siamese classification - Hadamard product

In [9]:
pt_siamese_hadamard_results = evaluate_datasets(chembl_datasets, "pt_siamese_hadamard", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_siamese_hadamard/CHEMBL234_Ki/checkpoints/epoch=11-step=444.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_siamese_hadamard/CHEMBL2047_EC50/checkpoints/epoch=3-step=28.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


## Siamese classification - Euclidean distance

In [10]:
pt_siamese_euclidean_results = evaluate_datasets(chembl_datasets, "pt_siamese_euclidean", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_siamese_euclidean/CHEMBL234_Ki/checkpoints/epoch=11-step=444.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_siamese_euclidean/CHEMBL2047_EC50/checkpoints/epoch=3-step=28.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


## Siamese classification - Cosine similarity

In [11]:
pt_siamese_cosine_results = evaluate_datasets(chembl_datasets, "pt_siamese_cosine", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_siamese_cosine/CHEMBL234_Ki/checkpoints/epoch=0-step=37.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_siamese_cosine/CHEMBL2047_EC50/checkpoints/epoch=3-step=28.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


## Siamese autoencoder - Naive two pass BCE, symmetric loss

In [12]:
pt_siamese_ae_naive_results = evaluate_datasets(chembl_datasets, "pt_siamese_ae_naive", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_siamese_ae_naive/CHEMBL234_Ki/checkpoints/epoch=3-step=148.ckpt
Evaluating CHEMBL234_Ki (classification)


INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_siamese_ae_naive/CHEMBL2047_EC50/checkpoints/epoch=18-step=133.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


## Siamese autoencoder SiamACLoss

In [13]:
pt_siamese_ae_siamac_results = evaluate_datasets(chembl_datasets, "pt_siamese_ae_siamac", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_siamese_ae_siamac/CHEMBL234_Ki/checkpoints/epoch=22-step=851.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_siamese_ae_siamac/CHEMBL2047_EC50/checkpoints/epoch=56-step=399.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


## Siamese autoencoder SiamACLoss all positives

In [14]:
pt_siamese_ae_siamac_allpos_results = evaluate_datasets(chembl_datasets, "pt_siamese_ae_siamac_allpos", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_siamese_ae_siamac_allpos/CHEMBL234_Ki/checkpoints/epoch=12-step=481.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_siamese_ae_siamac_allpos/CHEMBL2047_EC50/checkpoints/epoch=22-step=161.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


## Siamese autoencoder - negative cosine similarity

In [15]:
pt_siamese_ae_ncs_results = evaluate_datasets(chembl_datasets, "pt_siamese_ae_ncs", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_siamese_ae_ncs/CHEMBL234_Ki/checkpoints/epoch=7-step=296.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_siamese_ae_ncs/CHEMBL2047_EC50/checkpoints/epoch=7-step=56.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


## Siamese autoencoder - negative cosine similarity all positives

In [16]:
pt_siamese_ae_ncs_allpos_results = evaluate_datasets(chembl_datasets, "pt_siamese_ae_ncs_allpos", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_siamese_ae_ncs_allpos/CHEMBL234_Ki/checkpoints/epoch=6-step=259.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_siamese_ae_ncs_allpos/CHEMBL2047_EC50/checkpoints/epoch=7-step=56.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


## SimSiam

In [17]:
pt_simsiam_results = evaluate_datasets(chembl_datasets, "pt_simsiam", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_simsiam/CHEMBL234_Ki/checkpoints/epoch=3-step=148.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_simsiam/CHEMBL2047_EC50/checkpoints/epoch=3-step=28.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


## SimSiam all positives

In [18]:
pt_simsiam_allpos_results = evaluate_datasets(chembl_datasets, "pt_simsiam_allpos", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_simsiam_allpos/CHEMBL234_Ki/checkpoints/epoch=12-step=481.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_simsiam_allpos/CHEMBL2047_EC50/checkpoints/epoch=12-step=91.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


## Print to file

In [19]:
results_dict = {
    "pt_joint_1024": pt_joint_1024_results,
    "pt_joint_recon_1024": pt_joint_recon_1024_results,
    "pt_siamese_manhattan": pt_siamese_manhattan_results,
    "pt_siamese_hadamard" : pt_siamese_hadamard_results,
    "pt_siamese_euclidean" : pt_siamese_euclidean_results,
    "pt_siamese_cosine" : pt_siamese_cosine_results,
    "pt_siamese_ae_naive" : pt_siamese_ae_naive_results,
    "pt_siamese_ae_siamac" : pt_siamese_ae_siamac_results,
    "pt_siamese_ae_siamac_allpos" : pt_siamese_ae_siamac_allpos_results,
    "pt_siamese_ae_ncs": pt_siamese_ae_ncs_results,
    "pt_siamese_ae_ncs_allpos": pt_siamese_ae_ncs_allpos_results,
    "pt_simsiam": pt_simsiam_results,
    "pt_simsiam_allpos": pt_simsiam_allpos_results,
}

In [20]:
plot_data_dir = os.path.join("data", "plot")
if not os.path.exists(plot_data_dir):
    os.makedirs(plot_data_dir)

file_path = os.path.join(plot_data_dir, "ac_results.csv")

write_results_to_csv(results_dict, file_path)

In [21]:
results_dict

{'pt_joint_1024': {'CHEMBL234_Ki_classification': {'AUROC': 0.7999243341404358},
  'CHEMBL234_Ki_regression': {'RMSE': 1.4922264723317678,
   'Cliff_RMSE': 1.3639693715118844},
  'CHEMBL2835_Ki_classification': {'AUROC': 0.8434309053778081},
  'CHEMBL2835_Ki_regression': {'RMSE': 1.2417865393394536,
   'Cliff_RMSE': 1.3911546233667227},
  'CHEMBL4203_Ki_classification': {'AUROC': 0.9428733031674209},
  'CHEMBL4203_Ki_regression': {'RMSE': 1.2991894876018129,
   'Cliff_RMSE': 1.2506423498540493},
  'CHEMBL4616_EC50_classification': {'AUROC': 0.8273142382731424},
  'CHEMBL4616_EC50_regression': {'RMSE': 1.0606587810240862,
   'Cliff_RMSE': 0.9416473887710757},
  'CHEMBL264_Ki_classification': {'AUROC': 0.8411290826203709},
  'CHEMBL264_Ki_regression': {'RMSE': 1.4316886213055784,
   'Cliff_RMSE': 1.3858425380839736},
  'CHEMBL4792_Ki_classification': {'AUROC': 0.7121350364963503},
  'CHEMBL4792_Ki_regression': {'RMSE': 1.5597281763286779,
   'Cliff_RMSE': 1.5490887197581582},
  'CHEMBL20

## Refs

[1] Derek van Tilborg, Alisa Alenicheva, and Francesca Grisoni.“Exposing the Limitations of Molecular Machine Learning with Activity Cliffs”. In: Journal of Chemical Information and Modeling 62.23 (Dec. 2022), pp. 5938–5951. DOI: 10.1021/acs.jcim.2c01073. URL: https://doi.
org/10.1021/acs.jcim.2c01073.   
[2] César Miguel Valdez Córdova. Towards learning activity cliff-aware molecular representations. Publication pending.